# 2. Crée une Pipeline

## 1. Théory

Le dataset est composer de deux types de données, les données numérique et cathégoriques.
Ces deux types de données doivent etre traiter differament avant d'etre utilisable par un modèle de machine learning.

### 1. Numérique

Les données numérique peuvent avoire en générale quelques valeurs manquantes.
Ces données manquantes sont génantes pour entraier un model, il est donc important de résoudre ce problème
Pour concerver un nombre conéquent de données, nous allons compléter ces données avec un `SimpleInputer`.
L'alternative compléter les données serais de supprimer toutes les entrer contenant une ou plus donnée(s) manquante(s) ce qui réduirais le jeu de donnée de facon trop conséquente.
Pour compléter le jeu de données, nous allons utiliser la médiane de la meme données des autres entrer. Cela va nous permettre d'entrainer le model sur des données qui aurrons un meilleur impacte qu'une moyenne.

Apres avoir compléter les données manquantes, il peut etre intéressent des les rendre a la meme échelle.
Cette opération est importante pour les models comme le `KNeighborsClassifier` et est inutile pour les model de type arbres comme le `RandomForestClassifier`.
Comme cette étape ne prend pas beacoup de temps, qu'il peut etre au mieu néssessaire et au pire inutile, nous allon l'inclure dans la pipeline.

### 2. Cathégoriques

Les données cathégoriques peuvent aussie avoire des valeurs manquantes mais au fait que ces données ne sont pas numérique, il est impossible d'en faire une médiane.
On choisis ici aussi un `SimpleInputer` avec une stratégie contante c'est a dire que les données manquantes serons remplacer par la valeur la plus présente.

Apres avoir compléter les données manquantes, il faut convertir les données cathégoriques en données numérique pour qu'elle puissent etre utiliser par un model.
Pour faire cela, nous allons utliser un `OneHotEncoder` qui va nous permettre de faire cette transformation.

## 2. Pratique

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

main_dataset = pd.read_excel("./buffer/1-main_dataset.xlsx")
main_dataset.drop(columns=main_dataset.columns[0], axis=1, inplace=True)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

numeric_features = main_dataset.select_dtypes(include=['int64', 'float64']).columns
categorical_features = main_dataset.select_dtypes(include=['object']).columns

# Numeric specific pipeline
numeric_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='median')),
      ('scaler', StandardScaler())])

# Categorical specifique pipeline
categorical_transformer = Pipeline(steps=[
      ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
      ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Spliter between the numerical and categorical pipeline
preprocessor = ColumnTransformer(transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

# Create the pipeline for preprossessing and classifying
data_prepared = preprocessor.fit_transform(main_dataset)
new_column = list(numeric_features) + list(preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names(categorical_features))

df_data_prepared = pd.DataFrame(data=data_prepared, columns=new_column)

df_data_prepared.head()

/home/mathieu/Documents/ProjetIA/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Age,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,1.541369,-0.393938,-0.891688,-0.961486,1.405136,-0.678464,-1.150554,-0.932014,-1.322079,2.483396,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.648668,0.099639,-1.868426,-0.961486,-0.491661,-1.079486,2.129306,0.241988,-0.678877,0.155707,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,-0.539166,0.963398,1.061787,1.749610,2.725053,-0.678464,-0.057267,2.589994,-0.807517,-0.620189,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0.117845,-0.887515,2.038524,0.845911,0.386301,0.123580,-1.150554,2.589994,0.221606,1.707500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-0.539166,0.099639,-1.868426,-0.961486,-0.884109,0.524602,-0.877232,1.415991,-0.292956,-0.620189,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [3]:
df_data_prepared.to_excel("./buffer/2-main_dataset_prepared.xlsx")